In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv('train copy.csv')

In [ ]:
train_data.shape

In [ ]:
#According to https://github.com/shawnhero/ICDM2013-Expedia-Recommendation-System modify competitor data
#Convert comp_inv 
for i in range(1,9):
    train_data['comp'+str(i)+'_inv'].fillna(0, inplace=True)
    train_data['comp'+str(i)+'_inv'][train_data['comp'+str(i)+'_inv']==1] = 10
    train_data['comp'+str(i)+'_inv'][train_data['comp'+str(i)+'_inv']==-1] = 1
    train_data['comp'+str(i)+'_inv'][train_data['comp'+str(i)+'_inv']==0] = -1
    train_data['comp'+str(i)+'_inv'][train_data['comp'+str(i)+'_inv']==10] = 0
    
train_data['comp_inv_sum'] = train_data['comp1_inv']
for i in range(2,9):
    train_data['comp_inv_sum'] += train_data['comp'+str(i)+'_inv']

In [ ]:
#Convert compi_rate to comp_rate_sum
for i in range(1,9):
    train_data['comp'+str(i)+'_rate'].fillna(0, inplace=True)
train_data['comp_rate_sum'] = train_data['comp1_rate']
for i in range(2,9):
    train_data['comp_rate_sum'] += train_data['comp'+str(i)+'_rate']

In [ ]:
#drop competitor
cols = np.arange(27, 51, dtype=np.int)
train_data.drop(train_data.columns[cols],axis=1,inplace=True)

In [ ]:
#drop date_time as it's irrelevant
train_data.drop(['date_time'],axis=1,inplace=True)

In [ ]:
#Replace na in orig_destination_distance with negative number : -100
train_data['orig_destination_distance'].fillna(-100,inplace=True)

In [ ]:
#prop_location_score1&prop_location_score2:positively correlated, the higher the score, the hotel is more desirable
#drop prop_location_score2, rename prop_location_score1 as prop_location_score
train_data.drop(['prop_location_score2'],axis=1,inplace=True)
train_data.rename(columns={"prop_location_score1": "prop_location_score"},inplace=True)

In [ ]:
#Replace na in prop_review_score with negative number : -1
train_data['prop_review_score'].fillna(-1,inplace=True)

In [ ]:
#Create match and mismatch columns for visitor_hist_data
train_data['visitor_hist_adr_usd _diff'] = abs(train_data['visitor_hist_adr_usd'] - train_data['price_usd'])
train_data['visitor_hist_starrating_diff'] = abs(train_data['visitor_hist_starrating'] - train_data['prop_starrating'])
train_data['visitor_hist_starrating_diff'].fillna(-1,inplace=True)
train_data['visitor_hist_adr_usd _diff'].fillna(-1,inplace=True)
train_data.drop(['visitor_hist_starrating', 'visitor_hist_adr_usd'],axis=1,inplace=True)

In [ ]:
train_data.drop(['srch_query_affinity_score','gross_bookings_usd'],axis=1,inplace=True)

In [ ]:
#combine click_bool&booking_bool
train_data["click_book"] = train_data["click_bool"]+train_data["booking_bool"]*4
train_data.drop(['click_bool','booking_bool'],axis=1,inplace=True)

In [ ]:
train_data['price_sort']=train_data['price_usd'].groupby(train_data['srch_id']).rank(ascending=0,method='dense')

In [ ]:
train_data['current_price_diff'] = abs(np.exp(train_data['prop_log_historical_price'])-train_data['price_usd'])

In [ ]:
#use srch_id 70%train 30%test
import random
uniq_srch_id = list(train_data['srch_id'].unique())
res = list(random.sample(uniq_srch_id, round(len(uniq_srch_id)*0.7)))
train_train = train_data.loc[train_data['srch_id'].isin(res)]
train_test = train_data.loc[~train_data['srch_id'].isin(res)]

In [ ]:
train_train.to_csv('train_train.csv')
train_test.to_csv('train_test.csv')

In [ ]:
#check correlation with click_book 
corr = train_data.corr()
corr["click_book"]

In [ ]:
#mutual information
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html
from sklearn.feature_selection import mutual_info_classif
mut_info = mutual_info_classif(train_data.iloc[:,0:21], train_data[["click_book"]],discrete_features=True)

In [ ]:
mut_info.round(decimals=6)

In [ ]:
#Exploratory Data Analysis
srch_id_count = train_data.groupby(['srch_id']).count()['click_book']
srch_id_count.mean()
#For each srch_id avg appears 25 times.
srch_id_sum = train_data.groupby(['srch_id']).sum()['click_book']
sum(srch_id_sum>5)/len(srch_id_sum)
#booking rate only 4.35%
sum(srch_id_sum>1)/len(srch_id_sum)
#click rate 71.47%
sum(train_data["promotion_flag"])/len(train_data["promotion_flag"])
#promotion rate 21.588%
train_data[train_data["click_book"] == 5]["position"].mean()
#booking position mean 7.726